In [ ]:
import cv2

import numpy as np
import tensorflow as tf
import seaborn as sns
import tensorflow_io as tfio

import matplotlib.pyplot as plt

from keras.models import load_model

from tensorflow.keras import layers
from tensorflow.keras import models

from sklearn.utils import shuffle

import IPython.display as ipd
import librosa.display
from IPython import display

# Set the seed value for experiment reproducibility.
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

# Load Files

In [ ]:
batch_size = 32
sampling_size = 16000

train_data_ES_A = 'C:/<enter_file_path_here>'
test_data_ES_A = 'C:/<enter_file_path_here>'

train_ds_ES_A = tf.keras.utils.audio_dataset_from_directory(
    directory = train_data_ES_A,
    batch_size = batch_size,
    seed = 0,
    output_sequence_length = sampling_size,
    )

label_names_ES_A = np.array(train_ds_ES_A.class_names)
print()
print("label names:", label_names_ES_A)

train_ds_ES_A.element_spec

In [ ]:
val_ds_ES_A = tf.keras.utils.audio_dataset_from_directory(
    directory = test_data_ES_A,
    batch_size = batch_size,
    seed = 0,
    output_sequence_length=sampling_size,
    )


label_names_ES_A = np.array(train_ds_ES_A.class_names)
print()
print("label names:", label_names_ES_A)

# Combine dataset and labels

In [ ]:
def squeeze(audio, labels):
  audio = tf.squeeze(audio, axis=-1)
  return audio, labels

train_ds_ES_A = train_ds_ES_A.map(squeeze, tf.data.AUTOTUNE)
val_ds_ES_A = val_ds_ES_A.map(squeeze, tf.data.AUTOTUNE)

test_ds_ES_A = val_ds_ES_A.shard(num_shards=2, index=0)
val_ds_ES_A = val_ds_ES_A.shard(num_shards=2, index=1)

for example_audio_ES_A, example_labels_ES_A in train_ds_ES_A.take(1):  
  print(example_audio_ES_A.shape)
  print(example_labels_ES_A.shape)

# Feature Extraction

In [ ]:
param1 = 10
mels = 63

def get_spectrogram_ES_A(waveform, aug):
    spectrogram = tfio.audio.spectrogram(waveform, nfft=512, window=512, stride=256)

    mel_spectrogram = tfio.audio.melscale(spectrogram, rate=16000, mels=mels, fmin=0, fmax=8000)
    dbscale_mel_spectrogram = tfio.audio.dbscale(mel_spectrogram, top_db=80)

    # frequency masking
    #if aug == 1:
        #param1 = random.choice(list_a)
        #dbscale_mel_spectrogram = tfio.audio.freq_mask(dbscale_mel_spectrogram, param=param1)

    # time masking
    #elif aug == 2:
        #param2 = random.choice(list_a)
        #dbscale_mel_spectrogram = tfio.audio.time_mask(dbscale_mel_spectrogram, param=param2)
        
    # Add a `channels` dimension, so that the spectrogram can be used
    # as image-like input data with convolution layers (which expect
    # shape (`batch_size`, `height`, `width`, `channels`).
    mel_spectrograms = dbscale_mel_spectrogram[..., tf.newaxis]
     
    return mel_spectrograms

In [ ]:
def make_spec_ds_ES_A(ds, aug):
    return ds.map(
        map_func = lambda audio,label: (get_spectrogram_ES_A(audio, aug), label),
        num_parallel_calls=tf.data.AUTOTUNE)

train_spectrogram_ds_ES_A = make_spec_ds_ES_A(train_ds_ES_A, 0)
val_spectrogram_ds_ES_A = make_spec_ds_ES_A(val_ds_ES_A, 0)
test_spectrogram_ds_ES_A = make_spec_ds_ES_A(test_ds_ES_A, 0)

# Spec Augmentation
spec_a = 0

# add time mask
#train_spectrogram_ds_A_1 = make_spec_ds_ES_A(train_ds_ES_A, spec_a)

In [ ]:
for example_spectrograms_ES_A, example_spect_labels_ES_A in train_spectrogram_ds_ES_A.take(1):
  break

if spec_a == 1:
    for example_spectrograms_A_1, example_spect_labels_A_1 in train_spectrogram_ds_A_1.take(1):
      break

In [ ]:
if spec_a == 1:
    train_spectrogram_ds_ = train_spectrogram_ds_ES_A.concatenate(train_spectrogram_ds_A_1)
    train_spectrogram_ds_ES_A = train_spectrogram_ds_.cache().shuffle(10000).prefetch(tf.data.AUTOTUNE)

else:
    train_spectrogram_ds_ES_A = train_spectrogram_ds_ES_A.cache().shuffle(10000).prefetch(tf.data.AUTOTUNE)

val_spectrogram_ds_ES_A = val_spectrogram_ds_ES_A.cache().prefetch(tf.data.AUTOTUNE)
test_spectrogram_ds_ES_A = test_spectrogram_ds_ES_A.cache().prefetch(tf.data.AUTOTUNE)

# Training Model

In [ ]:
def build_model(seed=42):
    tf.random.set_seed(seed)
    input_shape = example_spectrograms_ES_A.shape[1:]
    print('Input shape:', input_shape)
    #print(len(input_shape))
    num_labels = len(label_names_ES_A)
    #print('Total labels:', num_labels)

    # Instantiate the `tf.keras.layers.Normalization` layer.
    norm_layer = layers.Normalization()
    # Fit the state of the layer to the spectrograms
    # with `Normalization.adapt`.
    norm_layer.adapt(data=train_spectrogram_ds_ES_A.map(map_func=lambda spec, label: spec))

    model = models.Sequential([
        layers.Input(shape=input_shape),
        # Downsample the input.
        layers.Resizing(63, 63),
        # Normalize.
        norm_layer,
        # Deduce dimnsion by 1
        layers.Reshape((63,63)),
        layers.LSTM(128, return_sequences=False),#, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(32, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(num_labels),
    ])
    return model

# Build and Traing Model

In [ ]:
EPOCHS = 10

def build_and_train(optimizer, i):    
    new = 'model_'+i+'_ES_specA.keras'
    model = build_model()
    model.compile(optimizer=optimizer,
                  # Loss Function to minimize
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    print(model.summary())
    return model.fit(train_spectrogram_ds_ES_A,
                     validation_data=val_spectrogram_ds_ES_A,
                     epochs=EPOCHS,
                     callbacks=tf.keras.callbacks.EarlyStopping(verbose=1, patience=2),
                    ), model.save(new)

## RMSProp

In [ ]:
optimizer = tf.keras.optimizers.RMSprop()
hist_rmsprop_001, model_rmsprop_001 = build_and_train(optimizer, 'RMSProp')

## Adam

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001, beta_1 = 0.9, beta_2 = 0.999)
hist_adam_001, model_adam_001 = build_and_train(optimizer, 'Adam')

## AdamW

In [ ]:
optimizer = tf.keras.optimizers.AdamW(
    learning_rate=0.001,
    weight_decay=0.004,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07)
    #tf.keras.optimizers.SGD(learning_rate = 0.01, momentum = 0.9)
hist_adamw_001, model_adamw_001 = build_and_train(optimizer, 'adamw')

## Adamax

In [ ]:
optimizer = tf.keras.optimizers.Adamax(learning_rate = 0.001, beta_1 = 0.9, beta_2 = 0.999)
hist_adamax_001, model_adamax_001 = build_and_train(optimizer, 'Adamax')

## Adagrad

In [ ]:
optimizer = tf.keras.optimizers.Adagrad(learning_rate=0.01)
hist_adagrad_001, model_adagrad_001 = build_and_train(optimizer, 'AdaGrad')

## Nadam

In [ ]:
optimizer = tf.keras.optimizers.Nadam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)
hist_nadam_001, model_nadam_001 = build_and_train(optimizer, 'Nadam')

## Nesterov

In [ ]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
hist_nesterov_001, model_nesterov_001 = build_and_train(optimizer, 'Nesterov')

## SGD

In [ ]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)
hist_sgd_001, model_sgd_001 = build_and_train(optimizer, 'sgd')

## Momentum

In [ ]:
optimizer = tf.keras.optimizers.SGD(learning_rate = 0.01, momentum = 0.9)
hist_momentum_001, model_momentum_001 = build_and_train(optimizer, 'momentum')

# Create Graphs for Accuracy and Loss

In [ ]:
for loss in ("loss", "val_loss"):
    params = {'axes.labelsize': 20,
              'axes.titlesize': 20,}
    plt.rcParams.update(params)
    plt.figure(figsize=(12, 8))
    opt_names = "SGD Momentum Nesterov AdaGrad RMSProp Adam AdamW Adamax Nadam"
    for hist, opt_name in zip((hist_sgd_001, hist_momentum_001, hist_nesterov_001,
                               hist_adagrad_001, hist_rmsprop_001, hist_adam_001,
                               hist_adamw_001, hist_adamax_001, hist_nadam_001),
                               opt_names.split()):
        plt.plot(hist.history[loss], label=f"{opt_name}", linewidth=3)

    plt.grid()
    plt.xlabel("Epochs")
    plt.ylabel({"loss": "Training loss", "val_loss": "Validation loss"}[loss])
    plt.legend(loc="upper right", fontsize=18)
    plt.axis([0, 10, 0.00001, 2.4])
    plt.show()

In [ ]:
for accuracy in ("accuracy", "val_accuracy"):
    plt.figure(figsize=(12, 8))
    opt_names = "SGD Momentum Nesterov AdaGrad RMSProp Adam AdamW Adamax Nadam"
    for hist, opt_name in zip((hist_sgd_001, hist_momentum_001, hist_nesterov_001,
                               hist_adagrad_001, hist_rmsprop_001, hist_adam_001,
                               hist_adamw_001, hist_adamax_001, hist_nadam_001),
                               opt_names.split()):
        plt.plot(hist.history[accuracy], label=f"{opt_name}", linewidth=3)

    plt.grid()
    plt.xlabel("Epochs")
    plt.ylabel({"accuracy": "Training acc", "val_accuracy": "Validation acc"}[accuracy])
    plt.legend(loc="lower right", fontsize=18)
    plt.axis([0.2, 10, 0, 1.01])
    plt.show()

# Load Models

In [ ]:
model_001 = load_model('model_RMSProp_ES_specA.keras')
model_002 = load_model('model_Adam_ES_specA.keras')
model_003 = load_model('model_AdamW_ES_specA.keras')
model_004 = load_model('model_Adamax_ES_specA.keras')
model_005 = load_model('model_AdaGrad_ES_specA.keras')
model_006 = load_model('model_Nadam_ES_specA.keras')
model_007 = load_model('model_Nesterov_ES_specA.keras')
model_008 = load_model('model_sgd_ES_specA.keras')
model_009 = load_model('model_momentum_ES_specA.keras')

## Evaluate Models

In [ ]:
rmsprop_accuracy0001 = model_001.evaluate(test_spectrogram_ds_ES_A, return_dict=True)
adam_accuracy0001 = model_002.evaluate(test_spectrogram_ds_ES_A, return_dict=True)
adamW_accuracy0001 = model_003.evaluate(test_spectrogram_ds_ES_A, return_dict=True)
adamax_accuracy0001 = model_004.evaluate(test_spectrogram_ds_ES_A, return_dict=True)
adagrad_accuracy0001 = model_005.evaluate(test_spectrogram_ds_ES_A, return_dict=True)
nadam_accuracy0001 = model_006.evaluate(test_spectrogram_ds_ES_A, return_dict=True)
nesterov_accuracy0001 = model_007.evaluate(test_spectrogram_ds_ES_A, return_dict=True)
sgd_accuracy0001 = model_008.evaluate(test_spectrogram_ds_ES_A, return_dict=True)
momentum_accuracy0001 = model_009.evaluate(test_spectrogram_ds_ES_A, return_dict=True)
print()
print(f'RMSProp:  {rmsprop_accuracy0001}')
print(f'Adam:     {adam_accuracy0001}')
print(f'AdamW:    {adamW_accuracy0001}')
print(f'Adamax:   {adamax_accuracy0001}')
print(f'AdaGrad:  {adagrad_accuracy0001}')
print(f'Nadam:    {nadam_accuracy0001}')
print(f'Nesterov: {nesterov_accuracy0001}')
print(f'SGD:      {sgd_accuracy0001}')
print(f'Momentum: {momentum_accuracy0001}')

## Predictions

In [ ]:
# Labels
y_true = tf.concat(list(test_spectrogram_ds_ES_A.map(lambda s,lab: lab)), axis=0)

# RMSProp
y_pred_rmsprop2 = model_001.predict(test_spectrogram_ds_ES_A)
y_pred_rms2 = tf.argmax(y_pred_rmsprop2, axis=1)

# Adam
y_pred_adam2 = model_002.predict(test_spectrogram_ds_ES_A)
y_pred_a2 = tf.argmax(y_pred_adam2, axis=1)

# AdamW
y_pred_adamw2 = model_002.predict(test_spectrogram_ds_ES_A)
y_pred_aw2 = tf.argmax(y_pred_adamw2, axis=1)

# Adamax
y_pred_adamax2 = model_004.predict(test_spectrogram_ds_ES_A)
y_pred_adax2 = tf.argmax(y_pred_adamax2, axis=1)

# AdaGrad
y_pred_adagrad2 = model_005.predict(test_spectrogram_ds_ES_A)
y_pred_ag2 = tf.argmax(y_pred_adagrad2, axis=1)

# Nadam
y_pred_nadam2 = model_006.predict(test_spectrogram_ds_ES_A)
y_pred_na2 = tf.argmax(y_pred_nadam2, axis=1)

# Nesterov
y_pred_nesterov2 = model_007.predict(test_spectrogram_ds_ES_A)
y_pred_ne2 = tf.argmax(y_pred_nesterov2, axis=1)

# SGD
y_pred_sgd2 = model_008.predict(test_spectrogram_ds_ES_A)
y_pred_sg2 = tf.argmax(y_pred_sgd2, axis=1)

# Momentum
y_pred_momentum2 = model_009.predict(test_spectrogram_ds_ES_A)
y_pred_mo2 = tf.argmax(y_pred_momentum2, axis=1)

## Confusion Matrices

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix

In [ ]:
def c_matrix(cm, title_opt3):
    optimizer_used3 = title_opt3
    # Normalise
    cmn = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    fig, ax = plt.subplots(figsize=(10,8))
    sns.heatmap(cmn, annot=True, fmt='.2f', xticklabels = label_names_ES_A, yticklabels = label_names_ES_A)
    plt.title(f'Optimizer Used: {optimizer_used3}')
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.show(block=False)

## RMSProp

In [ ]:
cm3 = confusion_matrix(y_true, y_pred_rms2)
c_matrix(cm3, 'RMSProp')

## Adam

In [ ]:
cm_adam3 = confusion_matrix(y_true, y_pred_a2)
c_matrix(cm_adam3, 'Adam')

## AdamW

In [ ]:
cm_adamw3 = confusion_matrix(y_true, y_pred_aw2)
c_matrix(cm_adamw3, 'AdamW')

## Adamax

In [ ]:
cm_adamax3 = confusion_matrix(y_true, y_pred_adax2)
c_matrix(cm_adamax3, 'Adamax')

## AdaGrad

In [ ]:
cm_adagrad3 = confusion_matrix(y_true, y_pred_ag2)
c_matrix(cm_adagrad3, 'AdaGrad')

## Nadam

In [ ]:
cm_nadam3 = confusion_matrix(y_true, y_pred_na2)
c_matrix(cm_nadam3, 'Nadam')

## Nesterov

In [ ]:
cm_nesterov3 = confusion_matrix(y_true, y_pred_ne2)
c_matrix(cm_nesterov3, 'Nesterov')

## SGD

In [ ]:
cm_sgd3 = confusion_matrix(y_true, y_pred_sg2)
c_matrix(cm_sgd3, 'SGD')

## Momentum

In [ ]:
cm_momentum3 = confusion_matrix(y_true, y_pred_mo2)
c_matrix(cm_momentum3, 'Momentum')